# Week 3
#### Reading the cleaned data from the csv
And assuming the data is cleaned already, we won't check anything

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../week2/cleaned_playstore.csv", index_col=0)
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,1.900000e+07,10000,Free,0.0,Everyone
1,Coloring book moana,ART_AND_DESIGN,3.9,967,1.400000e+07,500000,Free,0.0,Everyone
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.700000e+06,5000000,Free,0.0,Everyone
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,2.500000e+07,50000000,Free,0.0,Teen
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.800000e+06,100000,Free,0.0,Everyone


### Task 1
I will replot the previous results with different libraries. <br>
I already know how to create plots in matplotlib and Seaborn so I am going to use Altair 

In [3]:
import altair as alt

Make the data useful for viz

In [4]:
unique_values = data['Category'].unique()
unique_values.sort()

categories = pd.DataFrame({'Category': unique_values})

installs = data.groupby('Category')['Installs'].sum().reset_index()
categories['Total_Installs'] = installs['Installs']

prices = data.groupby('Category')['Price'].sum().reset_index()
categories['Total_Price'] = prices['Price']

data['Revenue'] = data['Price'] * data['Installs']
revenues = data.groupby('Category')['Revenue'].sum().reset_index()
categories['Total_Revenue'] = revenues['Revenue']

#### Task 2 for Week 2

In [5]:
# We declare the source, chart type in the first line then we declare the axes and the way we want to sort inside using alt.X
chart = alt.Chart(categories).mark_bar().encode(
    x= alt.X('Category:N', sort= '-y'),
    y = 'Total_Installs:Q'
)

# To modify X axis we use configure_axisX 
chart = chart.configure_axisX(
    labelAngle = 45
)

# To add header text we use properties
chart = chart.properties(
    title={
        "text": "Total Installs by Category"
    }
)

# To increase font size of the title text we use configure_title
chart = chart.configure_title(
    fontSize = 20
)

chart

alt.Chart(...)

#### Task 3 for Week 2 but horizontal

In [6]:
# We declare the source, chart type in the first line then we declare the axes and the way we want to sort inside using alt.X
chart = alt.Chart(categories).mark_bar().encode(    
    x = 'Total_Price:Q',
    y = alt.Y('Category:N', sort= '-x')   
)

# To modify X axis we use configure_axisX 
chart = chart.configure_axisX(
    labelAngle = 45
)

# To add header text we use properties
chart = chart.properties(
    title={
        "text": "Total Installs by Category"
    }
)

# To increase font size of the title text we use configure_title
chart = chart.configure_title(
    fontSize = 20
)

chart

alt.Chart(...)

#### Task 4 for Week 2 but it's a pie chart for top 5

In [7]:
df_sorted = categories.sort_values('Total_Revenue', ascending=False)[:5]

base = alt.Chart(df_sorted).encode(
    theta = alt.Theta('Total_Revenue:Q', stack=True),
    color = alt.Color('Category:N', legend = None),
    tooltip = ['Category', 'Total_Revenue']
).properties(
    width = 500,
    height = 500,
    title = {
        "text": 'Top 5 Total Revenue by Category'
    } 
)

pie = base.mark_arc(outerRadius = 200)
text = base.mark_text(radius = 230, size = 12).encode(text= 'Category:N')
pie+text

alt.LayerChart(...)